# Geosphere Austria tutorial
This tutorial walks through the use of a point data class for accessing station data in Austria
A video tutorial of this notebook can be found [here](https://www.loom.com/share/fcced7c00c47400e9a2b8b7b017949e5?sid=6a4cc255-48d0-48f0-8e14-a8aed0ef4bb3)

In [ ]:
# import metloom class https://data.hub.geosphere.at/dataset/
from metloom.pointdata import GeoSphereHistPointData

In [ ]:
# import packages
import geopandas as gpd
import pandas as pd
import fiona
import matplotlib.pyplot as plt
fiona.drvsupport.supported_drivers['kml'] = 'rw'
fiona.drvsupport.supported_drivers['KML'] = 'rw'

In [ ]:
# Install mapclassify and folium for gdf.explore()
!pip install mapclassify
!pip install folium

In [ ]:
# read and explore shape
gdf = gpd.read_file("./data/geosphere_test.kml")
gdf.explore()

In [ ]:
# Define variable of interest https://github.com/M3Works/metloom/blob/main/metloom/variables.py
variable = GeoSphereHistPointData.ALLOWED_VARIABLES.TEMP
# Get the points within our region of interest
points = GeoSphereHistPointData.points_from_geometry(gdf, [variable])
print(len(points))

In [ ]:
# Explore the points we located
points_df = points.to_dataframe()
points_df = points_df.set_crs("EPSG:4326")

points_df.explore()

In [ ]:
# define start and end date of interest
start = pd.to_datetime("2023-01-01")
end = pd.to_datetime("2023-01-11")

In [ ]:
# dataframe for storing combined data
final_df = pd.DataFrame()

# loop through first 15 points and store the data
for p in points.points[:15]:
    result = p.get_daily_data(start, end, [variable])
    if result is None:
        print(f"{p.name} did not return data")
    else:
        # reset the index to just be datetime
        result = result.reset_index().set_index("datetime")
        # store off the data in the final dataframe
        final_df[p.name + p.id] = result[variable.name]
final_df

In [ ]:
# Plot the timeseries
final_df.plot(ylabel=variable.name)